In [5]:
import pandas as pd
import geopandas as gpd
import unicodedata
import os


# Función para normalizar texto (quitar tildes y mayúsculas)
def normalize(text):
    if pd.isnull(text):
        return ""
    text = unicodedata.normalize("NFKD", str(text))
    return "".join(c for c in text if not unicodedata.combining(c)).strip().upper()


# Rutas
DATA_DIR = "data/"  # Ajusta según tu estructura
GJSON_RURAL = os.path.join(DATA_DIR, "parroquiasRurales.geojson")
GJSON_URB = os.path.join(DATA_DIR, "parroquiasUrbanas.geojson")
NBI_PATH = os.path.join(DATA_DIR, "NBI.xlsx")  # o .csv si aplica

# Leer parroquias rurales y urbanas
gdf_rurales = gpd.read_file(GJSON_RURAL).rename(columns={"DPA_DESPAR": "nombre"})
gdf_urbanas = gpd.read_file(GJSON_URB).rename(columns={"dpa_despar": "nombre"})

gdf_rurales["tipo"] = "rural"
gdf_urbanas["tipo"] = "urbana"

# Unir ambas en un solo GeoDataFrame
gdf_parroquias = pd.concat(
    [
        gdf_rurales[["nombre", "tipo"]],
        gdf_urbanas[["nombre", "tipo"]],
    ],
    ignore_index=True,
)

# Normalizar nombres del GeoJSON
gdf_parroquias["nombre_norm"] = gdf_parroquias["nombre"].apply(normalize)

# Leer Excel de NBI y normalizar nombres
df_nbi = pd.read_excel(NBI_PATH)
df_nbi["Parroquia_norm"] = df_nbi["Parroquia"].apply(normalize)

# Hacer merge
df_merge = gdf_parroquias.merge(
    df_nbi[["Parroquia", "Parroquia_norm"]],
    left_on="nombre_norm",
    right_on="Parroquia_norm",
    how="left",
)

# Ver parroquias sin correspondencia
parroquias_sin_match = df_merge[df_merge["Parroquia"].isna()][["nombre", "tipo"]]
print(f"Total sin correspondencia: {len(parroquias_sin_match)}")
parroquias_sin_match

Total sin correspondencia: 0


,nombre,tipo
